# Amazon Fine Food 

### data : https://www.kaggle.com/snap/amazon-fine-food-reviews



## 0. 구글 설정


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## 1. 모듈 불러오기

In [ ]:
!pip install beautifulsoup4

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords

# 2. 데이터 살펴보기


In [ ]:
# 구글
# df = pd.read_csv('Reviews.csv')

# 캐글
df = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv')

df.head()

### feature

* id : 인덱스
* ProductId : 제품의 아이디
* UserId : 고객의 아이디
* ProfileName : 고객의 프로필 이름
* HelpfulnessNumerator : 해당 리뷰가 도움이 되었다고 답한 고객의 수
* HelpfulnessDenominator : 해당 리뷰를 본 고객의 수
* Score : 리뷰 점수
* Time : 리뷰를 남긴 타임스탬프, 유닉스 시간을 사용하는 것 같다.
* Summary : 리뷰 요약
* Text : 리뷰 내용

In [ ]:
df.columns

In [ ]:
df.info() 

# ProfileName, Summary에서 null 값이 존재한다. 
# 우리가 필요한 건 Score와  Text 데이터이므로 제거하지 않음

### 중복값 및 잘못된 자료 제거

In [ ]:
sum(df.duplicated(subset={"UserId","ProfileName","Time","Text"})) # 중복존재확인

In [ ]:
# 중복값 제거
df.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=True)
df.shape

In [ ]:
df=df[df["HelpfulnessNumerator"]<=df["HelpfulnessDenominator"]] #분자>분모인 잘못된 자료 제거
df.shape

In [ ]:
df = df[['Score','Text']] #필요한 데이터만 남김

# NULL데이터 없음
print(df['Score'].isnull().sum())
df['Text'].isnull().sum()

### 리뷰 내용이 같으나 score가 다른 리뷰가 있는지 확인

In [ ]:
df["Text"].nunique()              # unique한 리뷰 개수

In [ ]:
len(df.groupby(["Score","Text"]))  # 점수 별로 리뷰를 그룹지었을 때 리뷰 수

리뷰 내용은 같지만 score가 다른 리뷰가 존재하므로 해당 리뷰는 score의 평균을 구함

In [ ]:
df = df.groupby('Text').mean().reset_index()
df['Score'].value_counts()

### Score에 따라 binary data로 변경 

In [ ]:
# Score가 3이하인 경우 0, 아닌 경우 1

df["Score"] = df["Score"].apply(lambda x : 1 if x>3 else 0)
df.head()

데이터 imbalance 확인

In [ ]:
df['Score'].value_counts()

In [ ]:
plt.figure(figsize = (10,7))
sns.countplot(x = df['Score'])
plt.title("Bar plot of Score")

# 3. 텍스트 전처리



1. decontract
2. html tags 제거
3. 알파벳만 유지 & 소문자로 변환 & split
4. stop words 제거 & Lemmatization(표제어추출)

### sample을 통해 어떻게 전처리가 이루어지는지 단계별로 확인함(skip해도 됨)

In [ ]:
sample = df.loc[0,'Text']
sample

In [ ]:
# decontract (baseline 코드에서 가져옴)

def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

sample = decontract(sample) # hadn't -> had not
sample

In [ ]:
# html tags 제거
sample = BeautifulSoup(sample, 'lxml').get_text()
sample

In [ ]:
# 알파벳만 유지, 소문자로 변환한 후 토큰화
sample = re.sub('[^A-Za-z]+', ' ', sample)
sample_token = sample.lower().split(" ")    
print(sample_token)

In [ ]:
# stop words 제거 & Lemmatization(표제어추출)
lemmatizer = WordNetLemmatizer()
stop_words= set(stopwords.words("english"))
stop_words.remove("not")                     # not은 제거하지 않음
    
sample = [lemmatizer.lemmatize(token, "v") for token in sample_token if not token in stop_words] 
print(sample) # not이 유지됨, my, i, this와 같은 불용어가 제거됨

In [ ]:
sample = " ".join(sample) # 한 문장으로 만듦
sample.strip()            # 양쪽 공백 제거

### 이제 모든 과정을 함수로 만들어서 실제 Text 데이터에 적용

In [ ]:
def preprocess_text(review):
      
    # decontract
    review = decontract(review) 
    
    # html tags 제거
    review = BeautifulSoup(review, 'lxml').get_text()
    
    # 알파벳만 유지 & 소문자로 변환 & split
    review = re.sub('[^A-Za-z]+', ' ', review)          
    review_token = review.lower().split(" ")                             
    
    # stop words 제거 & Lemmatization(표제어추출)
    lemmatizer = WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))
    stop_words.remove("not")                    # not은 제거하지 않음
    
    review = [lemmatizer.lemmatize(token, "v") for token in review_token if not token in stop_words] 
    
    review = " ".join(review)
    review.strip()
    
    return review

In [ ]:
df['Text'] = df['Text'].apply(lambda x: preprocess_text(x))
df.head() # 결과

## 전처리 자료 저장

In [ ]:
df.to_csv('df.csv')